# Checks in Okareo: An Introduction

<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/checks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## ðŸŽ¯ Goals

After using this notebook, you will be able to:
- Access the list of available `checks` in Okareo
- Generate and upload a custom `check` to Okareo
- Use `checks` to assess the behaviors of registered models in Okareo

First, import the Okareo library and use your [API key](https://docs.okareo.com/docs/guides/environment#setting-up-your-okareo-environment) to authenticate. You will also need an [OpenAI API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key).

In [ ]:
from okareo import Okareo

OKAREO_API_KEY = "<YOUR_OKAREO_API_KEY>"
OPENAI_API_KEY = "<YOUR_OPENAI_API_KEY>"

okareo = Okareo(OKAREO_API_KEY)

## Uploading a Scenario

Here we use an existing `.jsonl` file to create a seed scenario with the `upload_scenario_set` method. The data here includes short questions about a fictitious company called "WebBizz."

In [ ]:
import os
import requests

file_path_articles = "webbizz_retrieval_questions.jsonl"
scenario_name_articles = "WebBizz Retrieval Questions"

def load_or_download_file(file_path, scenario_name):
    try:
        # load the file to okareo
        source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name=scenario_name)
    except:
        print(f"- Loading file {file_path} to Okareo failed. Temporarily download the file from GitHub...") 

        # if the file doesn't exist, download it
        file_url = f"https://raw.githubusercontent.com/okareo-ai/okareo-python-sdk/main/examples/{file_path}"
        response = requests.get(file_url)
        with open(file_path, "wb") as f:
            f.write(response.content)

        # load the file to okareo
        source_scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name=scenario_name)

        # delete the file
        os.remove(file_path)
    return source_scenario

source_scenario  = load_or_download_file(file_path_articles, scenario_name_articles)
print(f"{scenario_name_articles}: {source_scenario.app_link}")

## Register a Model

For this notebook, we will register a simple model that makes the scenario `input` more concise.

In [ ]:
import random
import string

from okareo.model_under_test import OpenAIModel

random_string = ''.join(random.choices(string.ascii_letters, k=5))

mut_name = f"OpenAI Concise"
eval_name = f"OpenAI Concise Test Run"

USER_PROMPT_TEMPLATE = "{scenario_input}"
BREVITY_CONTEXT_TEMPLATE = """
Rewrite the following text in a more concise manner:
"""

print(f"Registering model...")
# Register the model to use in the test run
model_under_test = okareo.register_model(
name=mut_name,
model=OpenAIModel(
    model_id="gpt-3.5-turbo",
    temperature=0,
    system_prompt_template=BREVITY_CONTEXT_TEMPLATE,
    user_prompt_template=USER_PROMPT_TEMPLATE,
),
)

print(f"Model registered: {model_under_test}")

## Pre-defined Checks

To bootstrap your LLM evaluation workflow, Okareo offers pre-defined checks. Let's list the available checks with `okareo.get_all_checks()`.

In [ ]:
all_checks = okareo.get_all_checks()
all_checks_names = [check.name for check in all_checks]
all_checks_names

To see if our model is making the input more concise, let's use the `conciseness` and `levenshtein_distance_input` checks.

We can get some more details on these by running the following snippet:

In [ ]:
checks = ['conciseness', 'levenshtein_distance_input']
for check in all_checks:
    if check.name in checks:
        print(f"--- {check.name} ---")
        print(check)

The checks above can be used when calling `run_test` on a model under test using the `checks` parameter.

In [ ]:
from okareo_api_client.models.test_run_type import TestRunType

# Run the evaluation
evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=source_scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
    checks=checks,
)
print(f"See results in Okareo: {evaluation.app_link}")
metrics = evaluation.model_metrics

## Generating a Custom Check

In addition to the Okareo's predefined checks, you can use Okareo to generate custom checks.

In [ ]:
from okareo_api_client.models import EvaluatorSpecRequest

description = (
    "Calculate the number of tokens in the model_output divided by the number of tokens in the scenario_input."
    "Get the number of tokens by doing a simple `len().split(' ')` call on model_output and scenario_input."
)
output_data_type = "float"

generate_request = EvaluatorSpecRequest(
    description=description,
    requires_scenario_input=True,
    requires_scenario_result=False,
    output_data_type=output_data_type
)
generated_test = okareo.generate_check(generate_request).generated_code
print(generated_test)

## Uploading the check to Okareo



First, add the check to your local directory.

In [ ]:
check_name = 'token_compression_ratio'
file_path = f"{check_name}.py"
with open(file_path, "w+") as file:
    file.write(generated_test)

Then, import the Check class and call Okareo's create_or_update_check method

In [ ]:
from token_compression_ratio import Check

token_cr_check = okareo.create_or_update_check(
    name=check_name,
    description=description,
    check=Check()
)

## Use the uploaded check

Finally, we can use the uploaded check by adding it to the `checks` parameter of `run_test`.

In [ ]:
# Run the evaluation with the custom check
evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=source_scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
    checks=['conciseness', 'levenshtein_distance_input', check_name],
)
print(f"See results in Okareo: {evaluation.app_link}")
metrics = evaluation.model_metrics